<a href="https://colab.research.google.com/github/YUNJONGLYUL/-section4-project-/blob/main/Untitled12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# **SentenceBERT 모델 로드**

In [2]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')#트랜스 포머 모델

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)


[[-0.37510434 -0.7733843   0.59277123 ...  0.5792351   0.3268346
  -0.6508964 ]
 [-0.09361713 -0.18191518 -0.19230811 ... -0.03165795  0.30412516
  -0.2679362 ]]


데이터셋 로드
웰니스 대화 스크립트 데이터셋

In [3]:
from google.colab import files
myfile = files.upload()

Saving 02)웰니스_대화_스크립트_데이터셋.xlsx to 02)웰니스_대화_스크립트_데이터셋 (1).xlsx
Saving ChatbotData.csv to ChatbotData (1).csv
Saving 웰니스_대화_스크립트_데이터셋.xlsx to 웰니스_대화_스크립트_데이터셋 (1).xlsx


In [4]:
df1 =  pd.read_excel('웰니스_대화_스크립트_데이터셋.xlsx')
df2 =  pd.read_excel('02)웰니스_대화_스크립트_데이터셋.xlsx')
df3 = pd.read_csv('ChatbotData.csv')

In [5]:
df1.head()

구분                                               유저  \
0  감정/감정조절이상  제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1  감정/감정조절이상                         더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2  감정/감정조절이상                      하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3  감정/감정조절이상                                꼭 롤러코스터 타는 것 같아요.   
4  감정/감정조절이상                       롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.   

                                        챗봇  
0          감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.  
1                   저도 그 기분 이해해요. 많이 힘드시죠?  
2  그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.  
3                                      NaN  
4                                      NaN

In [6]:
df2.head()

핵심증상    intent keyword(임상키워드) 특이사항  \
0  지속되는우울한기분(우울감)  정신증상/우울감             우울  NaN   
1             NaN       NaN            NaN  NaN   
2             NaN       NaN            NaN  NaN   
3             NaN       NaN            NaN  NaN   
4             NaN       NaN            NaN  NaN   

                                                연관표현 utterance  \
0  머리가 짓눌러지는 느낌/머리가 맑지 않다/침울하면서 잠도 못 자다/후회가 많고 침울...       우울해   
1                                                NaN    너무 우울해   
2                                                NaN    우울해죽겠어   
3                                                NaN   기분이 우울해   
4                                                NaN     우울모드임   

              utterance(2차)   \
0                   임신해서 우울해   
1              아이 가지고 나서 우울해   
2    아이 가졌는데 기분 하나도 안 좋고 울적해   
3  임신했는데 남편이 하나도 안 챙겨줘서 우울하다   
4   진단 결과 안 좋게 나올 것 같아서 우울해    

                                        response(공감)  \
0     기분이 우울하시군요. 00님에게 스트레스 받는 일이 있었던 건 아닌지 걱정스러워요.   
1     마음이 우울해서 힘드셨군요. 00님의 소식이 궁금했는데 마침 저를 잘 찾아주셨네요.   
2                          00님이 울적하시다니 저도 마음이 안 좋아요.   
3  얼굴에서 00님의 기분이 가라앉은 것이 느껴져요. 큰일이 있으신 건 아니었으면 좋겠어요.   
4                                                NaN   

                                  임상질문그룹(연세의료원제공) utterance(긍정) utterance(부정)  \
0  거의 매일, 하루 중 대부분 시간을 우울한 기분으로 있거나 가라앉았던 적이 있나요?             응            아니   
1                                             NaN             어          아닌데?   
2                                             NaN             엉             노   
3                                             NaN            예스          아니거든   
4                                             NaN             네           아니야   

                                    긍정에 대한 챗봇 답변  \
0              저런… (우는 이모티콘) 기분이 나아지지 않는 상태군요.     
1                     힘든 상황이시군요. 저랑 더 얘기를 나눠볼까요?   
2  한번 가라앉은 기분을 다시 돌리기는 참 어려운 것 같아요. 저와 계속 얘기해봐요.   
3                                            NaN   
4                                            NaN   

                                      부정에 대한 챗봇 답변  추가발화(190917)  \
0  그렇군요. 기분이 계속 처지신다면 편안한 음악을 한번 들어보는 것도 좋을 것 같아요.           NaN   
1              그랬군요. 또 기분이 안 좋아지면 언제든 저에게 이야기해주세요.           NaN   
2     그런 증상은 없으셨군요. 00님의 머릿속에 낀 먹구름이 얼른 걷혔으면 좋겠어요.           NaN   
3                                              NaN           NaN   
4                                              NaN           NaN   

   추가발화 (191031)  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN

In [7]:
df3.head()

Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0

# 데이터 전처리

In [8]:
df1.columns

Index(['구분', '유저', '챗봇'], dtype='object')

In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5231 entries, 0 to 5230
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구분      5231 non-null   object
 1   유저      5231 non-null   object
 2   챗봇      1034 non-null   object
dtypes: object(3)
memory usage: 122.7+ KB


In [10]:
df1 = df1[~df1['챗봇'].isna()]#챗봇에 대답이 있는 것만 남기기기

df1.head()

구분                                                 유저  \
0     감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1     감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2     감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
15  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
16  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                            챗봇  
0              감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.  
1                       저도 그 기분 이해해요. 많이 힘드시죠?  
2      그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.  
15  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.  
16         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.

In [11]:
df1# 몇개가 남았는지 확인

구분                                                 유저  \
0       감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1       감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2       감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
15    감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
16    감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   
...           ...                                                ...   
5196     치료이력/응급실  그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...   
5197     치료이력/응급실                   파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…   
5213    현재상태/증상악화                     지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.    
5214    현재상태/증상악화                                   진짜 심해진 거 같긴 해요.    
5215    현재상태/증상악화                                 그런데 증상이 나빠진 거 같아.    

                                              챗봇  
0                감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.  
1                         저도 그 기분 이해해요. 많이 힘드시죠?  
2        그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.  
15    화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.  
16           정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.  
...                                          ...  
5196               응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?  
5197     응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.  
5213                    상태가 더 안 좋아지셨군요. 걱정이 되네요.  
5214                   정말 힘드시겠어요. 지금도 증상이 심하신가요?  
5215                  너무 심하시면 병원을 다시 가보는 건 어떨까요?  

[1034 rows x 3 columns]

In [12]:
df1.loc[0,'유저']#유저가 말한 대화 값을 보기기

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

유저 대화내용 인코딩

In [13]:
df1['embedding'] = pd.Series([[]] * len(df1)) #새로운 컬럼에 저장하기

df1['embedding'] = df1['유저'].map(lambda x: list(model.encode(x)))

df1.head()

구분                                                 유저  \
0     감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1     감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2     감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
15  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
16  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                            챗봇  \
0              감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                       저도 그 기분 이해해요. 많이 힘드시죠?   
2      그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
15  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
16         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                            embedding  
0   [-0.4806067, -0.2948689, 0.43789992, -0.640137...  
1   [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  
2   [-0.66520035, -0.081268325, 1.0945568, 0.10579...  
15  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  
16  [-0.20277731, -0.37413913, 0.04053215, -0.8624...

In [19]:
#나중에 사용할 수 도 있으니 전처리한 데이터를 파일로 만들기
df1.to_csv('wellness_dataset.csv', index=False)

# 챗봇 샘플

In [20]:
text = '요즘 힘들다'
embedding = model.encode(text)

In [21]:
df1['유사도'] = df1['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df1.head()

구분                                                 유저  \
0     감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1     감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2     감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
15  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
16  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                            챗봇  \
0              감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                       저도 그 기분 이해해요. 많이 힘드시죠?   
2      그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
15  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
16         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                            embedding       유사도  
0   [-0.4806067, -0.2948689, 0.43789992, -0.640137...  0.353954  
1   [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  0.422260  
2   [-0.66520035, -0.081268325, 1.0945568, 0.10579...  0.364640  
15  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  0.383667  
16  [-0.20277731, -0.37413913, 0.04053215, -0.8624...  0.309097

In [22]:
#코사인 유사도를 통해 가장 유사한 대답을 찾아 대답하기
answer = df1.loc[df1['유사도'].idxmax()]#idxmax()는 최대값 뽑기기

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['유사도'])

구분 감정/힘듦/지침
유사한 질문 너무 지친다…
챗봇 답변 누구나 같은 상황이 반복되거나 상황이 악화되면 지치기 마련이에요. 오늘 하루는 푹 쉬는 게 어떨까요?
유사도 0.7287181615829468


## df2와 df3를 df1과 결합해서 해보기기

In [23]:
#df2는 컬럼의 수가 너무 많고 이것을 어떻게 해야 연관지어서 할지 몰라 포기
df2.head()

핵심증상    intent keyword(임상키워드) 특이사항  \
0  지속되는우울한기분(우울감)  정신증상/우울감             우울  NaN   
1             NaN       NaN            NaN  NaN   
2             NaN       NaN            NaN  NaN   
3             NaN       NaN            NaN  NaN   
4             NaN       NaN            NaN  NaN   

                                                연관표현 utterance  \
0  머리가 짓눌러지는 느낌/머리가 맑지 않다/침울하면서 잠도 못 자다/후회가 많고 침울...       우울해   
1                                                NaN    너무 우울해   
2                                                NaN    우울해죽겠어   
3                                                NaN   기분이 우울해   
4                                                NaN     우울모드임   

              utterance(2차)   \
0                   임신해서 우울해   
1              아이 가지고 나서 우울해   
2    아이 가졌는데 기분 하나도 안 좋고 울적해   
3  임신했는데 남편이 하나도 안 챙겨줘서 우울하다   
4   진단 결과 안 좋게 나올 것 같아서 우울해    

                                        response(공감)  \
0     기분이 우울하시군요. 00님에게 스트레스 받는 일이 있었던 건 아닌지 걱정스러워요.   
1     마음이 우울해서 힘드셨군요. 00님의 소식이 궁금했는데 마침 저를 잘 찾아주셨네요.   
2                          00님이 울적하시다니 저도 마음이 안 좋아요.   
3  얼굴에서 00님의 기분이 가라앉은 것이 느껴져요. 큰일이 있으신 건 아니었으면 좋겠어요.   
4                                                NaN   

                                  임상질문그룹(연세의료원제공) utterance(긍정) utterance(부정)  \
0  거의 매일, 하루 중 대부분 시간을 우울한 기분으로 있거나 가라앉았던 적이 있나요?             응            아니   
1                                             NaN             어          아닌데?   
2                                             NaN             엉             노   
3                                             NaN            예스          아니거든   
4                                             NaN             네           아니야   

                                    긍정에 대한 챗봇 답변  \
0              저런… (우는 이모티콘) 기분이 나아지지 않는 상태군요.     
1                     힘든 상황이시군요. 저랑 더 얘기를 나눠볼까요?   
2  한번 가라앉은 기분을 다시 돌리기는 참 어려운 것 같아요. 저와 계속 얘기해봐요.   
3                                            NaN   
4                                            NaN   

                                      부정에 대한 챗봇 답변  추가발화(190917)  \
0  그렇군요. 기분이 계속 처지신다면 편안한 음악을 한번 들어보는 것도 좋을 것 같아요.           NaN   
1              그랬군요. 또 기분이 안 좋아지면 언제든 저에게 이야기해주세요.           NaN   
2     그런 증상은 없으셨군요. 00님의 머릿속에 낀 먹구름이 얼른 걷혔으면 좋겠어요.           NaN   
3                                              NaN           NaN   
4                                              NaN           NaN   

   추가발화 (191031)  
0            NaN  
1            NaN  
2            NaN  
3            NaN  
4            NaN

# df3의 Q와 A, 그리고 label를 df1과 같은 형식으로 만들기:



1.   구분, 유저,챗봇 순으로 컬럼 제목 재지정하기
2.   임베딩하고 유사도 구해서 더하기기





In [24]:
df3= df3[['label', 'Q', 'A']]#순서 바꾸기기
df3.head()

label                Q            A
0      0           12시 땡!   하루가 또 가네요.
1      0      1지망 학교 떨어졌어    위로해 드립니다.
2      0     3박4일 놀러가고 싶다  여행은 언제나 좋죠.
3      0  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.
4      0          PPL 심하네   눈살이 찌푸려지죠.

In [25]:
df3.columns = ['구분', '유저', '챗봇']#컬럼 명 변경하기기
df3.columns

Index(['구분', '유저', '챗봇'], dtype='object')

In [26]:
#구분을 일상으로 바꾸기
df3['구분'] = '일상/일상대화'
df3.head()

구분               유저           챗봇
0  일상/일상대화           12시 땡!   하루가 또 가네요.
1  일상/일상대화      1지망 학교 떨어졌어    위로해 드립니다.
2  일상/일상대화     3박4일 놀러가고 싶다  여행은 언제나 좋죠.
3  일상/일상대화  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.
4  일상/일상대화          PPL 심하네   눈살이 찌푸려지죠.

In [27]:
df3 = df3[~df3['챗봇'].isna()]#챗봇에 대답이 있는 것만 남기기기
df3.head()

구분               유저           챗봇
0  일상/일상대화           12시 땡!   하루가 또 가네요.
1  일상/일상대화      1지망 학교 떨어졌어    위로해 드립니다.
2  일상/일상대화     3박4일 놀러가고 싶다  여행은 언제나 좋죠.
3  일상/일상대화  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.
4  일상/일상대화          PPL 심하네   눈살이 찌푸려지죠.

In [28]:
df3# 몇개가 남았는지 확인

구분                       유저                        챗봇
0      일상/일상대화                   12시 땡!                하루가 또 가네요.
1      일상/일상대화              1지망 학교 떨어졌어                 위로해 드립니다.
2      일상/일상대화             3박4일 놀러가고 싶다               여행은 언제나 좋죠.
3      일상/일상대화          3박4일 정도 놀러가고 싶다               여행은 언제나 좋죠.
4      일상/일상대화                  PPL 심하네                눈살이 찌푸려지죠.
...        ...                      ...                       ...
11818  일상/일상대화           훔쳐보는 것도 눈치 보임.        티가 나니까 눈치가 보이는 거죠!
11819  일상/일상대화           훔쳐보는 것도 눈치 보임.             훔쳐보는 거 티나나봐요.
11820  일상/일상대화              흑기사 해주는 짝남.                    설렜겠어요.
11821  일상/일상대화  힘든 연애 좋은 연애라는게 무슨 차이일까?  잘 헤어질 수 있는 사이 여부인 거 같아요.
11822  일상/일상대화               힘들어서 결혼할까봐        도피성 결혼은 하지 않길 바라요.

[11823 rows x 3 columns]

유저 대화내용 인코딩

In [29]:
df3['embedding'] = pd.Series([[]] * len(df3)) #새로운 컬럼에 저장하기

df3['embedding'] = df3['유저'].map(lambda x: list(model.encode(x)))

In [30]:
#나중에 사용할 수 도 있으니 전처리한 데이터를 파일로 만들기
df3.to_csv('wellness_dataset_final.csv', index=False)

In [31]:
text = '놀러가고 싶다.'
embedding = model.encode(text)

In [32]:
df3['유사도'] = df1['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df3.head()

구분               유저           챗봇  \
0  일상/일상대화           12시 땡!   하루가 또 가네요.   
1  일상/일상대화      1지망 학교 떨어졌어    위로해 드립니다.   
2  일상/일상대화     3박4일 놀러가고 싶다  여행은 언제나 좋죠.   
3  일상/일상대화  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.   
4  일상/일상대화          PPL 심하네   눈살이 찌푸려지죠.   

                                           embedding       유사도  
0  [-0.19072628, 0.3214766, 0.5987375, 0.44617844...  0.214501  
1  [0.12320567, 0.6423684, 0.68536043, -0.4291238...  0.231537  
2  [0.024357699, -0.22393546, 0.22917677, -0.4501...  0.296373  
3  [0.03628567, -0.21048236, 0.07269404, -0.54036...       NaN  
4  [0.022030713, 0.70552164, -0.17157258, 0.16597...       NaN

In [33]:
#코사인 유사도를 통해 가장 유사한 대답을 찾아 대답하기
answer = df3.loc[df3['유사도'].idxmax()]#idxmax()는 최대값 뽑기기

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['유사도'])

구분 일상/일상대화
유사한 질문 스트레스 참기 어렵다
챗봇 답변 진정하세요.
유사도 0.49500736594200134


# **데이터 합치기**

In [34]:
df1.head()

구분                                                 유저  \
0     감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1     감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2     감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
15  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
16  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                            챗봇  \
0              감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                       저도 그 기분 이해해요. 많이 힘드시죠?   
2      그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
15  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
16         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                            embedding       유사도  
0   [-0.4806067, -0.2948689, 0.43789992, -0.640137...  0.353954  
1   [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  0.422260  
2   [-0.66520035, -0.081268325, 1.0945568, 0.10579...  0.364640  
15  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  0.383667  
16  [-0.20277731, -0.37413913, 0.04053215, -0.8624...  0.309097

In [35]:
df3.head()

구분               유저           챗봇  \
0  일상/일상대화           12시 땡!   하루가 또 가네요.   
1  일상/일상대화      1지망 학교 떨어졌어    위로해 드립니다.   
2  일상/일상대화     3박4일 놀러가고 싶다  여행은 언제나 좋죠.   
3  일상/일상대화  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.   
4  일상/일상대화          PPL 심하네   눈살이 찌푸려지죠.   

                                           embedding       유사도  
0  [-0.19072628, 0.3214766, 0.5987375, 0.44617844...  0.214501  
1  [0.12320567, 0.6423684, 0.68536043, -0.4291238...  0.231537  
2  [0.024357699, -0.22393546, 0.22917677, -0.4501...  0.296373  
3  [0.03628567, -0.21048236, 0.07269404, -0.54036...       NaN  
4  [0.022030713, 0.70552164, -0.17157258, 0.16597...       NaN

# 저장해 놓은 'wellness_dataset.csv','wellness_dataset_final.csv'을 사용해서 데이터 합치기

In [36]:
df_first = pd.read_csv('wellness_dataset.csv')
df_first.head()

구분                                                 유저  \
0    감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1    감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2    감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
4  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                           챗봇  \
0             감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                      저도 그 기분 이해해요. 많이 힘드시죠?   
2     그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
3  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
4         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                           embedding  
0  [-0.4806067, -0.2948689, 0.43789992, -0.640137...  
1  [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  
2  [-0.66520035, -0.081268325, 1.0945568, 0.10579...  
3  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  
4  [-0.20277731, -0.37413913, 0.04053215, -0.8624...

In [37]:
df_second = pd.read_csv('wellness_dataset_final.csv')
df_second.head()

구분               유저           챗봇  \
0  일상/일상대화           12시 땡!   하루가 또 가네요.   
1  일상/일상대화      1지망 학교 떨어졌어    위로해 드립니다.   
2  일상/일상대화     3박4일 놀러가고 싶다  여행은 언제나 좋죠.   
3  일상/일상대화  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.   
4  일상/일상대화          PPL 심하네   눈살이 찌푸려지죠.   

                                           embedding  
0  [-0.19072628, 0.3214766, 0.5987375, 0.44617844...  
1  [0.12320567, 0.6423684, 0.68536043, -0.4291238...  
2  [0.024357699, -0.22393546, 0.22917677, -0.4501...  
3  [0.03628567, -0.21048236, 0.07269404, -0.54036...  
4  [0.022030713, 0.70552164, -0.17157258, 0.16597...

In [38]:
result = pd.concat([df_first,df_second], ignore_index=True)
print(result)

                구분                                                 유저  \
0        감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1        감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2        감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3      감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
4      감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   
...            ...                                                ...   
12852      일상/일상대화                                     훔쳐보는 것도 눈치 보임.   
12853      일상/일상대화                                     훔쳐보는 것도 눈치 보임.   
12854      일상/일상대화                                        흑기사 해주는 짝남.   
12855      일상/일상대화                            힘든 연애 좋은 연애라는게 무슨 차이일까?   
12856      일상/일상대화                                         힘들어서 결혼할까봐   

                                               챗봇  \
0                 감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                

In [39]:
result.head()

구분                                                 유저  \
0    감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1    감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2    감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
4  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                           챗봇  \
0             감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                      저도 그 기분 이해해요. 많이 힘드시죠?   
2     그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
3  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
4         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                           embedding  
0  [-0.4806067, -0.2948689, 0.43789992, -0.640137...  
1  [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  
2  [-0.66520035, -0.081268325, 1.0945568, 0.10579...  
3  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  
4  [-0.20277731, -0.37413913, 0.04053215, -0.8624...

In [40]:
result = result[['구분','유저','챗봇']]
result

구분                                                 유저  \
0        감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1        감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2        감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3      감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
4      감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   
...            ...                                                ...   
12852      일상/일상대화                                     훔쳐보는 것도 눈치 보임.   
12853      일상/일상대화                                     훔쳐보는 것도 눈치 보임.   
12854      일상/일상대화                                        흑기사 해주는 짝남.   
12855      일상/일상대화                            힘든 연애 좋은 연애라는게 무슨 차이일까?   
12856      일상/일상대화                                         힘들어서 결혼할까봐   

                                               챗봇  
0                 감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.  
1                          저도 그 기분 이해해요. 많이 힘드시죠?  
2         그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.  
3      화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.  
4             정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.  
...                                           ...  
12852                          티가 나니까 눈치가 보이는 거죠!  
12853                               훔쳐보는 거 티나나봐요.  
12854                                      설렜겠어요.  
12855                    잘 헤어질 수 있는 사이 여부인 거 같아요.  
12856                          도피성 결혼은 하지 않길 바라요.  

[12857 rows x 3 columns]

In [41]:
#나중에 사용할 수 도 있으니 전처리한 데이터를 파일로 만들기
result.to_csv('wellness_dataset_finalfinal.csv', index=False)

In [42]:
#임베딩 값 저장
result['embedding'] = pd.Series([[]] * len(result)) #새로운 컬럼에 저장하기

result['embedding'] = result['유저'].map(lambda x: list(model.encode(x)))

result.head()

구분                                                 유저  \
0    감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1    감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2    감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
4  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                           챗봇  \
0             감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                      저도 그 기분 이해해요. 많이 힘드시죠?   
2     그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
3  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
4         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                           embedding  
0  [-0.4806067, -0.2948689, 0.43789992, -0.640137...  
1  [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  
2  [-0.66520035, -0.081268325, 1.0945568, 0.10579...  
3  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  
4  [-0.20277731, -0.37413913, 0.04053215, -0.8624...

In [43]:
result11 = result.copy()
result22 = result.copy()
result33 = result.copy()
result44 = result.copy()
result11.head()

구분                                                 유저  \
0    감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1    감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2    감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
4  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                           챗봇  \
0             감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                      저도 그 기분 이해해요. 많이 힘드시죠?   
2     그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
3  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
4         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                           embedding  
0  [-0.4806067, -0.2948689, 0.43789992, -0.640137...  
1  [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  
2  [-0.66520035, -0.081268325, 1.0945568, 0.10579...  
3  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  
4  [-0.20277731, -0.37413913, 0.04053215, -0.8624...

In [44]:
#나중에 사용할 수 도 있으니 전처리한 데이터를 파일로 만들기
result11.to_csv('wellness_dataset_final3.csv', index=False)

# 최종실험

1번실험

In [45]:
text1 = '난 왜그럴까'
embedding = model.encode(text1)

In [46]:
result11['유사도'] = result11['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

result11.head()

구분                                                 유저  \
0    감정/감정조절이상    제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.   
1    감정/감정조절이상                           더 이상 내 감정을 내가 컨트롤 못 하겠어.   
2    감정/감정조절이상                        하루종일 오르락내리락 롤러코스터 타는 기분이에요.   
3  감정/감정조절이상/화  평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...   
4  감정/감정조절이상/화                              예전보다 화내는 게 과격해진 거 같아.   

                                           챗봇  \
0             감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.   
1                      저도 그 기분 이해해요. 많이 힘드시죠?   
2     그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.   
3  화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.   
4         정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.   

                                           embedding       유사도  
0  [-0.4806067, -0.2948689, 0.43789992, -0.640137...  0.399846  
1  [-1.1561574, -0.1450624, 0.2949035, -0.6739493...  0.327278  
2  [-0.66520035, -0.081268325, 1.0945568, 0.10579...  0.162143  
3  [-0.7679059, 0.465207, 0.52850705, -0.5076043,...  0.349332  
4  [-0.20277731, -0.37413913, 0.04053215, -0.8624...  0.326883

In [47]:
#코사인 유사도를 통해 가장 유사한 대답을 찾아 대답하기
answer1 = result11.loc[result11['유사도'].idxmax()]#idxmax()는 최대값 뽑기기

print('구분', answer1['구분'])
print('유사한 질문', answer1['유저'])
print('챗봇 답변', answer1['챗봇'])
print('유사도', answer1['유사도'])

구분 일상/일상대화
유사한 질문 나 왜 이러지?
챗봇 답변 자책하지마세요.
유사도 0.9360513091087341


2번 실험

In [48]:
text2 = '지각이다'
embedding = model.encode(text2)

In [49]:
result22['유사도'] = result22['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

result22.head()
#코사인 유사도를 통해 가장 유사한 대답을 찾아 대답하기
answer2 = result22.loc[result22['유사도'].idxmax()]#idxmax()는 최대값 뽑기기

print('구분', answer2['구분'])
print('유사한 질문', answer2['유저'])
print('챗봇 답변', answer2['챗봇'])
print('유사도', answer2['유사도'])

구분 일상/일상대화
유사한 질문 이 지각쟁이
챗봇 답변 잠깐 핸드폰 게임 하시면 올거예요.
유사도 0.7920008897781372


3번 실험

In [50]:
text3 = '머리가 아파'
embedding = model.encode(text3)

In [51]:
result33['유사도'] = result33['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

result33.head()
#코사인 유사도를 통해 가장 유사한 대답을 찾아 대답하기
answer3 = result33.loc[result33['유사도'].idxmax()]#idxmax()는 최대값 뽑기기

print('구분', answer3['구분'])
print('유사한 질문', answer3['유저'])
print('챗봇 답변', answer3['챗봇'])
print('유사도', answer3['유사도'])

구분 일상/일상대화
유사한 질문 머리 아파
챗봇 답변 시원한 바람 쉬고 약먹기!
유사도 0.9524505138397217


4번 실험

In [52]:
text4 = '한숨이 늘었어'
embedding = model.encode(text4)

In [53]:
result44['유사도'] = result44['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

result44.head()
#코사인 유사도를 통해 가장 유사한 대답을 찾아 대답하기
answer4 = result44.loc[result44['유사도'].idxmax()]#idxmax()는 최대값 뽑기기

print('구분', answer4['구분'])
print('유사한 질문', answer4['유저'])
print('챗봇 답변', answer4['챗봇'])
print('유사도', answer4['유사도'])

구분 일상/일상대화
유사한 질문 한숨만 자꾸 나온다
챗봇 답변 다 잘 될 거예요.
유사도 0.7348341941833496


# 임베딩 값이 정확하지 않아서 자연어 처리 플랫폼 Pororo 사용

In [ ]:
result2 = result[['구분','유저','챗봇']]
result2.head()

In [ ]:
!pip install konlpy

In [ ]:
!pip install pororo

In [ ]:
!pip install python-mecab-ko

In [49]:
#필요패키지
from pororo import Pororo
import pandas as pd
from sentence_transformers import util
import torch
import numpy as np

ModuleNotFoundError: ignored

In [ ]:
#임베딩 진행 상황 보기 위한 tqdm 사용
from tqdm import tqdm
tqdm.pandas()

In [ ]:
#임베딩 되지 않는 데이터
result2.head()

In [ ]:
#임베딩 하기
se = Pororo(task="sentence_embedding", lang="ko")

result2['embedding_vectors'] = result2['유저'].progress_map(lambda x : se(x))

In [ ]:
#데이터 변환 -> 코사인 유사도를 구해주기 위해 미리 구해놓은 임베딩 데이터를 tensor로 변환환
embedding_data = torch.tensor(result2['embedding_vectors'].tolist())

# 인풋 데이터를 임베딩 하고 tensor 타입으로 변환

In [ ]:
def return_sim_question(input_sentence):
  input_sentence = se(input_sentence)
  input_sentence = torch.tensor(input_sentence)
  return input_sentence

# **챗봇 구현 함수**

In [ ]:
def chatbot(message):
  message = message.strip()
  # 입력 문장 임베딩
  embedding_sentence = return_sim_question(message)
  
  # 미리 구해진 임베딩 데이터와 현재 임베딩 데이터의 코사인 유사도 추출
  cos_sim = util.pytorch_cos_sim(embedding_sentence,embedding_data)
  cos_sim = cos_sim.cpu()
  
  #유사도가 가장 비슷한 질문 인덱스 반환
  best_sim_idx = int(np.argmax(cos_sim))
  
  #유사도가 가장 비슷한 질문에 해당하는 답변 제공
  answer = result2['챗봇'][best_sim_idx]
  return answer